# Задача и данные
Требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.
Наши данные - заголовки новостей, лейбл - является ли новость фейком <br>
Задача: по заголовку определить является ли новость фейком <br>

Первая часть: поиск с google search (лучшее по качеству) <br>
Вторая часть: NLP

In [ ]:
!pip install beautifulsoup4
!pip install google
!pip install tensorflow==1.15.2
!pip install deeppavlov

In [ ]:
import pandas as pd

train_df = pd.read_csv('./dataset/train.tsv', delimiter='\t')
test_df = pd.read_csv('./dataset/test.tsv', delimiter='\t')
train_df.head(5)

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


Возьмем новость фейк и не фейк и заметим, что по этим запросам находятся страницы в поиске. Взяв выборку примерно из 10 фейковых новостей можно увидеть что они взяты с сайта фейковых новостей panorama.pub <br>
Предполагаю, что лучшая модель для решения задачи не включает в себя статистических языковых моделей, а делается простым поиском

из интересных находок: 
* (xe-xe-xe) новость <br> Заголовок: Россияне обхитрили рост цен <br> URL: https://lenta.ru/news/2018/04/17/xe_xe_xe/ 

## Поиск гугл
Гугл дает ограничение по запросам, нужно выставлять искусственную задержку между запросами <br>
Пока гугл не кинул ошибку Too many requests на найденных 53 тренировочных новостях F1 мера показала 1.0 <br>
Этого достаточно чтобы утверждать, что эта модель будет или идеальной, или близка к идеальной <br>
т.к. один из критериев оценки - F1 мера точности, я включу эту модель в финальное решение

In [ ]:
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
from time import sleep

# titles = train_df['title'].values
titles = test_df['title'].values

def searchPredict(titles, labels=[]):
  for title in titles[len(labels):]:
      query = title
      # print(query)
      for j in search(query, tld="co.in", num=1, stop=10, pause=2):
          if 'panorama' in j:
              labels.append(1)
          else:
              labels.append(0)
          # print(j, labels[-1])
          sleep(1)
          break
  return labels

In [ ]:
# takes about 2 hours
labels = searchPredict(titles, labels)

In [ ]:
# train_df labels
len(labels)

53

In [ ]:
# train_df labels
from sklearn.metrics import f1_score
y_true = train_df['is_fake'].values[:len(labels)]
y_pred = labels
f1_score(y_true, y_pred, average='macro')

1.0

In [ ]:
test_df['is_fake'] = labels
test_df.to_csv('./predictions.tsv', index=False, sep='\t')

## Статистические языковые модели
Хорошо, мы смогли понять откуда данные и получить отличную F1 метрику, теперь интересная часть <br>
Смогут ли современные языковые модели по заголовку понять, фейковая ли новость? <br>
Посмотрев на данные своими глазами и попробовав решить эту задачу без помощи автоматики, могу сказать что я затрудняюсь сказать по заголовку новости фейк это или нет. <br>

**В этой задаче важно иметь subword токенизацию, так как именованные сущности встречаются почти в каждом заголовке**<br>
Считаю, что стоит попробовать такие подходы как: <br>
1. Векторизация TF-IDF (или любая другая токен-векторизовалка) + MLP (самый слабый из трех, потому что скорее всего не хватит Term'ов для редких аббревиатур, условно "ФННБ" встретится один раз и будет мало веса добавлять к вектору)
2. ELMO + classifier (хороший tradeoff качество/скорость)
3. BERT (лучшее по качеству) <br>

Считаю, что не стоит включать word2vec в исследование, т.к. корпус включает в себя примеры со многим кол-вом именованных существительных и все неизвестные имена и названия при токенизации w2v будут отмечены как *UNK*, а что в решении того, фейк новость или нет, это помешает<br>

### Data split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_df['title'].tolist(), train_df['is_fake'].tolist(), test_size=0.2, random_state=42)

### TF-IDF + classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text_clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', lo

In [ ]:
%%time
predicted = text_clf.predict(X_test)

CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 23.2 ms


In [ ]:
from sklearn.metrics import f1_score
y_true = y_test
y_pred = predicted
f1_score(y_true, y_pred, average='macro')

0.7819391183357655

### ELMO (WMT news)

In [ ]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [ ]:
train_splitted = [title.split() for title in X_train]
val_splitted = [title.split() for title in X_test]

In [ ]:
import pickle

val_vectorized = elmo(val_splitted)
print('valid data vectorization done')
train_vectorized = elmo(train_splitted)
print('train data vectorization done')

with open('./train_vectors.pickle', 'wb') as f:
    pickle.dump(train_vectorized, f)
with open('./valid_vectors.pickle', 'wb') as f:
    pickle.dump(val_vectorized, f)

# Если подгрузили вектора ELMO
# with open('./train_vectors.pickle', 'rb') as f:
#     train_vectorized = pickle.load(f)
# with open('./valid_vectors.pickle', 'rb') as f:
#     val_vectorized   = pickle.load(f)

valid data vectorization done
train data vectorization done


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MLP_clf(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


elmo_clf = MLP_clf()

In [ ]:
import torch
from torch.utils.data import Dataset

class ELMODataset(Dataset):
  def __init__(self, vectors, targets):
    self.vectors = vectors
    self.targets = targets

  def __len__(self):
    return len(self.vectors)

  def __getitem__(self, idx):
    vector = self.vectors[idx]
    target = self.targets[idx]

    return [torch.tensor(vector), torch.tensor(target, dtype=torch.long)]

In [ ]:
batch_size = 4

trainset = ELMODataset(train_vectorized, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = ELMODataset(val_vectorized, y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(elmo_clf.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(6):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = elmo_clf(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0

[1,   200] loss: 6.915
[1,   400] loss: 6.854
[1,   600] loss: 6.745
[1,   800] loss: 6.374
[1,  1000] loss: 5.148
[2,   200] loss: 3.129
[2,   400] loss: 2.997
[2,   600] loss: 2.807
[2,   800] loss: 2.957
[2,  1000] loss: 2.757
[3,   200] loss: 2.438
[3,   400] loss: 2.098
[3,   600] loss: 2.181
[3,   800] loss: 2.161
[3,  1000] loss: 2.099
[4,   200] loss: 1.818
[4,   400] loss: 1.675
[4,   600] loss: 2.070
[4,   800] loss: 2.091
[4,  1000] loss: 2.069
[5,   200] loss: 1.759
[5,   400] loss: 1.594
[5,   600] loss: 1.534
[5,   800] loss: 1.305
[5,  1000] loss: 1.662
[6,   200] loss: 0.867
[6,   400] loss: 1.348
[6,   600] loss: 1.104
[6,   800] loss: 1.295
[6,  1000] loss: 1.349


In [ ]:
predicted = []
for i, data in enumerate(testloader, 0):
    inputs, labels = data

    outputs = elmo_clf(inputs)
    predicted.extend(torch.argmax(outputs, dim=1).tolist())

In [ ]:
from sklearn.metrics import f1_score
y_true = y_test
y_pred = predicted
f1_score(y_true, y_pred, average='macro')

0.9114198871037777

In [ ]:
test_splitted = [title.split() for title in test_df['title']]

In [ ]:
%%time
test_vectorized = elmo(test_splitted)

CPU times: user 3min 20s, sys: 8.8 s, total: 3min 29s
Wall time: 1min 48s


In [ ]:
testset = ELMODataset(test_vectorized, test_df['is_fake'].values)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
%%time
predicted = []
for i, data in enumerate(testloader, 0):
    inputs, labels = data

    outputs = elmo_clf(inputs)
    predicted.extend(torch.argmax(outputs, dim=1).tolist())

CPU times: user 522 ms, sys: 256 ms, total: 778 ms
Wall time: 1.24 s


In [ ]:
test_df['is_fake'] = predicted
test_df.to_csv('/content/vvasin/elmo_predicted.tsv', index=False, sep='\t')

### BERT

In [ ]:
!pip install -r bertRequirements.txt

In [ ]:
from bert_classifier import BertClassifier

classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=5,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [ ]:
classifier.preparation(
        X_train=X_train,
        y_train=y_train,
        X_valid=X_test,
        y_valid=y_test
    )

In [ ]:
classifier.train()

Epoch 1/5
Train loss 0.6086610032398075 accuracy 0.8113330438558403
Val loss 0.5664233940816403 accuracy 0.8515625
----------
Epoch 2/5
Train loss 0.45336810380153286 accuracy 0.8894919669995658
Val loss 0.6560147800638434 accuracy 0.8559027777777777
----------
Epoch 3/5
Train loss 0.35123130670054403 accuracy 0.9205384281372123
Val loss 0.7322190685178308 accuracy 0.8541666666666666
----------
Epoch 4/5
Train loss 0.27132093387910833 accuracy 0.9411636995223621
Val loss 0.7267862384517988 accuracy 0.8637152777777777
----------
Epoch 5/5
Train loss 0.21875851424554246 accuracy 0.9546244029526705
Val loss 0.7532525329388591 accuracy 0.8628472222222222
----------


In [ ]:
%%time
predictions = [classifier.predict(t) for t in X_test]

CPU times: user 2min 36s, sys: 9.81 s, total: 2min 46s
Wall time: 2min 53s


In [ ]:
from sklearn.metrics import f1_score
y_true = y_test
y_pred = predictions
f1_score(y_true, y_pred, average='macro')

0.8635589020076448

# Итоги

1. Качество
* google search 1.0 F1 score
* TF-IDF + SGDclassifier 0.7819 F1 score 
* ELMO + MLP 0.9114 F1 score 
* tinyBERT 0.8636 F1 score <br>
2. Время отработки на тестовых данных (на CPU)
* google search >1 ч.
* TF-IDF + SGDclassifier 21.2 мс
* ELMO + MLP 3.5 минуты
* tinyBERT 3 минуты <br>

In [ ]:
#Ну все, тф-идф можно в продакшн